In [2]:
import re
import numpy as np


In [3]:
import csv

def sensitive_function(sentiment):
    if sentiment.lower() == "negative":
        return -1
    elif sentiment.lower() == "neutral":
        return 0
    elif sentiment.lower() == "positive":
        return 1
    else:
        raise ValueError("It is not a valid sentiment: {}".format(sentiment))

def get_labels_and_texts(file):
    labels = []
    texts = []
    with open(file, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            labels.append(sensitive_function(row['Sentiment']))
            texts.append(row['Sentence'])
    return labels, texts

data_labels, data_texts = get_labels_and_texts(r'C:\Users\91950\Desktop\sentiment_data.csv')



In [4]:
data_labels[0]

1

In [5]:
data_texts[0]

"The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model ."

In [6]:
print(data_labels)

[1, -1, 1, 0, 0, 1, -1, -1, 1, 0, 1, -1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, -1, -1, -1, -1, 1, 1, -1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, -1, 1, 1, -1, 0, 1, 0, 0, 0, 0, 0, -1, 0, 1, 0, 1, 0, -1, 0, 1, 0, -1, 0, 0, 0, 0, 1, 1, -1, 1, 0, 0, 0, 0, 0, -1, 0, 1, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 1, 0, 1, 0, 1, -1, 1, -1, 1, 0, 0, 0, 0, -1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, -1, 1, 1, 0, 1, 0, 1, -1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, -1, 0, 0, -1, 1, 0, 1, 0, -1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, -1, 0, 0, 1, 0, 1, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 1, 0, 0, -1, 0, 1, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, -1, 1, 1, -1, 1, 0, 1, 0, 0, 1, 0, 0, -1, 0, 1, -1, 1, 0, 1, 0, 1, 0, -1, 1, 1, -1, 0, -1, 0, -1, 0, 0, 1, 0, -1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, -1, 0, -1, 0, 0, 0, 0, 1, 0, 1, -1, 0, 1, 0, -1, 1, 1, -1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, -1, 0, 1, 0, 0, 1, 1, -1, 1, 1, 0, 0, 1, -1, 0, 0, 0, 1, 0, -1

In [7]:
len(data_texts)

5842

In [8]:
len(data_labels)

5842

In [9]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
data_texts = normalize_texts(data_texts)

In [10]:
data_texts[0]

'the geosolutions technology will leverage benefon  s gps solutions by providing location based search technology   a communities platform   location relevant multimedia content and a new and powerful commercial model  '

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

model = CountVectorizer(binary=True)
model.fit(data_texts)
x = model.transform(data_texts)

In [12]:
x

<5842x10440 sparse matrix of type '<class 'numpy.int64'>'
	with 93454 stored elements in Compressed Sparse Row format>

In [13]:
x[0]

<1x10440 sparse matrix of type '<class 'numpy.int64'>'
	with 23 stored elements in Compressed Sparse Row format>

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [15]:
np.random.seed(42)
train_text,test_text,train_label,test_label=train_test_split(x,data_labels,test_size=0.2)
model_classifier=RandomForestClassifier()

# SVC score

In [16]:
model_svc=SVC()
model_svc.fit(train_text,train_label)
model_svc.score(test_text,test_label)

0.6937553464499572

In [17]:
from sklearn.model_selection import cross_val_score

# SCV cross validation

In [18]:
scores=cross_val_score(model_svc,x,data_labels,cv=5)
scores.mean()*100

68.86331544347703

# RFC score

In [28]:
model_classifier.fit(train_text,train_label)
model_classifier.score(test_text,test_label)

0.6561163387510693

# RFC cross validation

In [30]:
score_classifier=cross_val_score(model_classifier,x,data_labels,cv=5)
score_classifier*100

array([63.81522669, 63.90076989, 62.84246575, 64.38356164, 64.81164384])

### over all SVC performs better than RFC

In [19]:
count_neg=data_labels.count(-1)
count_neg

860

In [20]:
count_neu=data_labels.count(0)
count_neu

3130

In [21]:
count_pos=data_labels.count(1)
count_pos

1852

In [22]:
len(data_labels)

5842

In [35]:
y_preds=model_classifier.predict(train_text)
y_preds

array([0, 0, 1, ..., 1, 1, 1])

In [36]:
predicted_list=np.array(y_preds)
predicted_list

array([0, 0, 1, ..., 1, 1, 1])